# Tutorial for MDC tool

In this notebook, it is presented a brief tutorial on how to define and train a small Convolutional Neural Network for the classification of the MNIST Dataset. At the end of the notebook, it will be showed how to convert the keras model into the QONNX format.

The MNIST database of handwritten digits, available from this page, has a training set of 60,000 examples, and a test set of 10,000 examples. It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image (https://yann.lecun.com/exdb/mnist/).

![alt text](images/mnist_eg.png "MNIST example")



Keras (https://keras.io/) is an open source free library that gives access to an interface for Neural Networks (NN) in Python. It is now integrated into the Tensorflow library.
With Keras we have the possibility of defining and training neural networks. QKeras (https://github.com/google/qkeras) is a quantization extension to Keras that provides drop-in replacement for some of the Keras layers, especially the ones that creates parameters and activation layers, and perform arithmetic operations, so that we can quickly create a deep quantized version of Keras network.


In this example we are going to explore the capabilities of Qkeras, by defining and training a Convolutional Neural Network.
First, we import the necessaries packages and do some checks on libraries versions 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import time
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
import os

Now we create a folder to store the outputs of this script.

In [ ]:
# Specify the folder name
folder_name = 'Mnist_Training'

script_path = os.getcwd()
# Get the current working directory
current_directory = os.path.dirname(script_path)

# Print the current working directory
print("Current working directory:", current_directory)


# Create the full path to the new folder
output_path = current_directory + "/" + folder_name

# Check if the folder already exists
if not os.path.exists(output_path):
    # Create the folder
    os.makedirs(output_path)
    print(f"Folder '{folder_name}' created successfully.")
else:
    print(f"Folder '{folder_name}' already exists.")

print(output_path)

It is time to load the MNIST dataset, and to extract information like training size (train_size), the input shape (input__shape) and the number of classes to classify (n_classes)

In [ ]:
ds_train, info = tfds.load('mnist', split='train[:90%]', with_info=True, as_supervised=True)
ds_test = tfds.load('mnist', split='test', shuffle_files=True, as_supervised=True)
ds_val = tfds.load('mnist', split='train[-10%:]', shuffle_files=True, as_supervised=True)

assert isinstance(ds_train, tf.data.Dataset)
train_size = int(info.splits['train'].num_examples)
input_shape = info.features['image'].shape
n_classes = info.features['label'].num_classes

print('Training on {} samples of input shape {}, belonging to {} classes'.format(train_size, input_shape, n_classes))
fig = tfds.show_examples(ds_train, info)

We define a function to apply some preprocessing to the dataset and we manage the training and validation sets

In [ ]:
def preprocess(image, label, nclasses=10):
    image = tf.cast(image, tf.float32) / 255.0
    label = tf.one_hot(tf.squeeze(label), nclasses)
    return image, label

In [ ]:
batch_size = 1024

train_data = ds_train.map(preprocess, n_classes)  # Get dataset as image and one-hot encoded labels, divided by max RGB
train_data = train_data.shuffle(4096).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

for example in train_data.take(1):
    break
print("X train batch shape = {}, Y train batch shape = {} ".format(example[0].shape, example[1].shape))

val_data = ds_val.map(preprocess, n_classes)
val_data = val_data.batch(batch_size)
val_data = val_data.prefetch(tf.data.experimental.AUTOTUNE)

# For  testing, we get the full dataset in memory as it's rather small.
# We fetch it as numpy arrays to have access to labels and images separately
X_test, Y_test = tfds.as_numpy(tfds.load('mnist', split='test', batch_size=-1, as_supervised=True))
X_test, Y_test = preprocess(X_test, Y_test, nclasses=n_classes)
print("X test batch shape = {}, Y test batch shape = {} ".format(X_test.shape, Y_test.shape))

Here, the model is defined. Each layer has its own kernel_quantizers and bias_quantizers, which are functions that apply the quantization to, respectively, weights and biases of the Conv and Gemm layers, and QActivations, which quantizes the activations. While all the Relu layers are quantized, even if with different precisions for each model, we always left untouched the last activation, the Sigmoid layer, to achieve a better accuracy.


In [ ]:
from keras.layers import *
from qkeras import *
from qkeras.qlayers import QDense, QActivation, quantized_bits, quantized_relu
from qkeras import QConv2D
from keras.models import Model
from tensorflow.keras.regularizers import l1


x = x_in = Input(shape=input_shape, name = "input_layer")

x = QConv2D(32, (3,3), name = "q_conv2d", padding='same',kernel_quantizer= quantized_bits(bits=8, integer=4, alpha=1),
    bias_quantizer= quantized_bits(bits=8, integer=4, alpha=1))(x)
x = QActivation(quantized_relu(bits=16, integer=8, use_sigmoid=0, negative_slope=0.0), name="act_1")(x)
x = MaxPooling2D(pool_size=(2,2),name = "max_pool_1")(x)
x = QConv2D(32, (3,3), name = "q_conv2d_1",padding='same',kernel_quantizer= quantized_bits(bits=4, integer=2, alpha=1),
    bias_quantizer= quantized_bits(bits=4, integer=2, alpha=1))(x)
x = QActivation(quantized_relu(bits=16, integer=8, use_sigmoid=0, negative_slope=0.0), name="act_2")(x)
x = MaxPooling2D(pool_size=(2,2), name = "max_pool_2")(x)
x = Flatten(name = "flatten")(x)
x = QDense((10),name = "q_dense",kernel_quantizer= quantized_bits(bits=8, integer=4, alpha=1),
    bias_quantizer= quantized_bits(bits=8, integer=4, alpha=1)) (x)   # num_classes = 10
x_out = Activation('sigmoid', name='output_sigmoid')(x)

qmodel = Model(inputs=[x_in], outputs=[x_out], name='qkeras')

qmodel.summary()

Here, the training phase can start. A low number of epochs is chosen as the model is fairly small and simple, leading to a low training time. We save both the whole model and separately, its weights.

In [ ]:
train = True

n_epochs = 3
if train:
    LOSS = tf.keras.losses.CategoricalCrossentropy()
    OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=3e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=True)
    qmodel.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=["accuracy"])

    callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=10, verbose=1),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
    ]

    start = time.time()
    history = qmodel.fit(train_data, epochs=n_epochs, validation_data=val_data, callbacks=callbacks, verbose=1)
    end = time.time()
    print('\n It took {} minutes to train!\n'.format((end - start) / 60.0))

    qmodel.save_weights("model_weights.h5")

    qmodel.save('model_1.h5')

At this point, the keras model can be converted into the QONNX format. The QONNX format is an exstension of ONNX, an open format built to represent machine learning models. ONNX defines a common set of operators - the building blocks of machine learning and deep learning models - and a common file format to enable AI developers to use models with a variety of frameworks, tools, runtimes, and compilers (https://onnx.ai/).

QONNX (Quantized ONNX), starting from ONNX, introduces three new custom operators, Quant, BipolarQuant, and Trunc, in order to represent arbitrary-precision uniform quantization in ONNX. This enables representation of binary, ternary, 3-bit, 4-bit, 6-bit or any other quantization (https://github.com/fastmachinelearning/qonnx).

In [ ]:
from qonnx.converters import from_keras

path = output_path + '/qonnx_model_1.onnx'
print("conversion to qonnx...")
qonnx_model, _  = from_keras(
    qmodel,
    name="qkeras_to_qonnx_converted",
    input_signature=None,
    opset=None,
    custom_ops=None,
    custom_op_handlers=None,
    custom_rewriter=None,
    inputs_as_nchw=None,
    extra_opset=None,
    shape_override=None,
    target=None,
    large_model=False,
    output_path = path,
)

To leverage the capabilities of the MDC tool of merging different CNN models into onereconfigurable accelerator, we take the original model, change the precision of one or more layers, and freeze the others during training while fine-tuning is applied to the changed layers. This enables MDC to reuse the common, unchanged layers. In this example, we changed the precision of the second Convolutional layer ("q_conv2d_1") from 4 to 8 bits.

In [ ]:
x = x_in = Input(shape=input_shape, name = "input_layer")

x = QConv2D(32, (3,3), name = "q_conv2d", padding='same',kernel_quantizer= quantized_bits(bits=8, integer=4, alpha=1),
    bias_quantizer= quantized_bits(bits=8, integer=4, alpha=1))(x)
x = QActivation(quantized_relu(bits=16, integer=8, use_sigmoid=0, negative_slope=0.0), name="act_1")(x)
x = MaxPooling2D(pool_size=(2,2),name = "max_pool_1")(x)
x = QConv2D(32, (3,3), name = "q_conv2d_1",padding='same',kernel_quantizer= quantized_bits(bits=8, integer=4, alpha=1),
    bias_quantizer= quantized_bits(bits=4, integer=2, alpha=1))(x)
x = QActivation(quantized_relu(bits=16, integer=8, use_sigmoid=0, negative_slope=0.0), name="act_2")(x)
x = MaxPooling2D(pool_size=(2,2), name = "max_pool_2")(x)
x = Flatten(name = "flatten")(x)
x = QDense((10),name = "q_dense",kernel_quantizer= quantized_bits(bits=8, integer=4, alpha=1),
    bias_quantizer= quantized_bits(bits=8, integer=4, alpha=1)) (x)   # num_classes = 10
x_out = Activation('sigmoid', name='output_sigmoid')(x)

new_qmodel = Model(inputs=[x_in], outputs=[x_out], name='qkeras')

new_qmodel.summary()

Here we load the weights of the original model and freeze every layer, expect the "q_conv2d_1" one. AFter this operation, we can start the training of this new model: only the "q_conv2d_1" will be able to change the value of its weights.

In [ ]:
new_qmodel.load_weights("model_weights.h5")

for layer in new_qmodel.layers:
    if layer.name != "q_conv2d_1":
        layer.trainable = False

In [ ]:
train = True

n_epochs = 3
if train:
    LOSS = tf.keras.losses.CategoricalCrossentropy()
    OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=3e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=True)
    new_qmodel.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=["accuracy"])

    callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=10, verbose=1),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
    ]

    start = time.time()
    history = new_qmodel.fit(train_data, epochs=n_epochs, validation_data=val_data, callbacks=callbacks, verbose=1)
    end = time.time()
    print('\n It took {} minutes to train!\n'.format((end - start) / 60.0))


    new_qmodel.save('model_2.h5')

We can double check if effecively the new model's weights are changed. After this chechk, we can convert also this new model into the QONNX format.

In [ ]:
# Lists to store the names of layers with matching and non-matching weights
matching_layers = []
non_matching_layers = []

for original_layer, modified_layer in zip(qmodel.layers, new_qmodel.layers):
    

    # Check if the layer has weights (some layers like Dropout do not)
    if original_layer.get_weights() and modified_layer.get_weights():
        # Compare weights
        weights_match = all(
            (original_weight == modified_weight).all()
            for original_weight, modified_weight in zip(original_layer.get_weights(), modified_layer.get_weights())
        )
        
        if weights_match:
            matching_layers.append(original_layer.name)
        else:
            non_matching_layers.append(original_layer.name)

print("Layers with matching weights:", matching_layers)
print("Layers with non-matching weights:", non_matching_layers)


In [ ]:
from qonnx.converters import from_keras

path = output_path + '/qonnx_model_2.onnx'
print("conversion to qonnx...")
qonnx_model, _  = from_keras(
    qmodel,
    name="qkeras_to_qonnx_converted",
    input_signature=None,
    opset=None,
    custom_ops=None,
    custom_op_handlers=None,
    custom_rewriter=None,
    inputs_as_nchw=None,
    extra_opset=None,
    shape_override=None,
    target=None,
    large_model=False,
    output_path = path,
)